#Trabajo Práctico 2
##    NLP
###Desarrollo: Antonio Peroni

##Instalar dependencias

In [1]:
!pip install llama_index==0.9.38

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.2 MB/s eta 0:00:00


In [2]:
!pip install gdown sentence-transformers pypdf2 langchain python-decouple chromadb spacy openai tensorflow tensorflow_text unidecode python-dotenv scikit-learn
!python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.5/149.5 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.0/817.0 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.7/521.7 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.4/246.4 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

##Recoleccion de datos

In [3]:
# Obtenemos los archivos que usaremos como fuente de conocimiento
import gdown
import os
import shutil

# Link datos instrumentacion
url = 'https://drive.google.com/drive/folders/1QYAFO54ELsdy3A8FTP0xW-2KtCIdHuXE?usp=sharing'
# Link datos comunicacion txt, dataframe stock, logs.env
url1 = 'https://drive.google.com/drive/folders/1vUZoxAyCeChrmgvmictQ65k8C4FWI32q?usp=sharing'

# Descarga carpeta
gdown.download_folder(url, quiet=True, output='data')
gdown.download_folder(url1, quiet=True, output='other')

# Crear la carpeta 'llamaindex_data' si no existe
carpeta_destino = 'llamaindex_data'
if not os.path.exists(carpeta_destino):
    os.makedirs(carpeta_destino)
destino = '/content'

# Mover todos los archivos de 'data' a 'llamaindex_data'
carpeta_origen = 'data'
for filename in os.listdir(carpeta_origen):
    ruta_origen = os.path.join(carpeta_origen, filename)
    ruta_destino = os.path.join(carpeta_destino, filename)
    shutil.move(ruta_origen, ruta_destino)
# Mover todos los archivos de 'data' a 'content'
carpeta_origen1 = 'other'
for filename in os.listdir(carpeta_origen1):
    ruta_origen1 = os.path.join(carpeta_origen1, filename)
    ruta_destino = os.path.join(destino, filename)
    shutil.move(ruta_origen1, ruta_destino)


# Eliminar la carpetas vacias
shutil.rmtree(carpeta_origen)
shutil.rmtree(carpeta_origen1)

print("Archivos movidos con éxito.")

Archivos movidos con éxito.


In [4]:
#KEY
from decouple import config
import openai
from dotenv import load_dotenv
load_dotenv()

openai.api_key =config("OPENAI_API_KEY")
api_key = config('HUGGINGFACE_TOKEN')

## Embedding

In [5]:
from langchain.embeddings import HuggingFaceEmbeddings
from llama_index.embeddings.langchain import LangchainEmbedding

models = HuggingFaceEmbeddings(model_name='sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
embeddings = LangchainEmbedding(models)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.13k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/402 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# iniciar el splitter y embeddings
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

##Base vectorial

In [32]:
#crear base vectorial
import os
import PyPDF2
import chromadb

# iniciar base como cliente
client = chromadb.PersistentClient(path="/content/")
collection = client.create_collection(name="instrumentacion")


In [33]:
import string

# convertir pdf a texto
def pdf_to_text(file_path):
    pdf_file = open(file_path, 'rb')
    pdf_reader = PyPDF2.PdfReader(pdf_file)
    text = ""
    for page_num in range( len(pdf_reader.pages)):
        text += pdf_reader.pages[page_num].extract_text()
    pdf_file.close()
    return text

for filename in os.listdir('/content/llamaindex_data'):
    if filename.endswith('.pdf'):
        # Convert PDF to text
        text = pdf_to_text(os.path.join('/content/llamaindex_data', filename))
        # Convertir a minúsculas
        text = text.lower()
       # Eliminar la puntuación utilizando translate()
        text = text.translate(str.maketrans('', '', string.punctuation))
        # Split text into chunks
        chunks = text_splitter.split_text(text)

        # Convert chunks to vector representations and store in Chroma DB
        documents_list = []
        embeddings_list = []
        ids_list = []

        for i, chunk in enumerate(chunks):
            vector = embeddings.get_text_embedding(chunk)

            documents_list.append(chunk)
            embeddings_list.append(vector)
            ids_list.append(f"{filename}_{i}")


        collection.add(
            embeddings=embeddings_list,
            documents=documents_list,
            ids=ids_list
        )


In [34]:
import chromadb
from langchain.embeddings import HuggingFaceEmbeddings

# Initialize Chroma DB client
client = chromadb.PersistentClient(path="/content/")
collection = client.get_collection(name="instrumentacion")

def query_chroma(query):
  # Convert query to vector representation
  query_vector = embeddings.get_text_embedding(query)

  # Query Chroma DB with the vector representation
  results = collection.query(query_embeddings=query_vector, n_results=2, include=["documents"] )

  # Print results
  for result in results["documents"]:
      for i in result:
          print(i)


In [35]:
print(query_chroma('¿que es histeresis?'))

descarga de los campos de leona y oritupano  tesis de grado udo 
 
artículos técnicos y minicursos de la revista isatech  de la isa
416  histéresis hysteresis 
 
la histéresis es la diferencia máxima que se observa en los valores indicados por el índice o la 
pluma del instrumento para el mismo valor cualqu iera del campo de medida cuando la variable 
recorre toda la escala en los dos sentidos ascendente y descendente 
se expresa en tanto por ciento del alcance de la medida por ejemplo si en el instrumento de la figura 43 c es de ± 03  su valor será de ± 03  de 200 °c  ± 06 °c en la figura 43 c 
pueden verse las curvas de histéresis que están dibujadas exageradamente para apreciar bien su forma hay que señalar que el término zona muerta está incluido dentro de la histéresis 
  
417  función de transferencia 
 
es la relación matemática gráfica o tabular entre las expresiones función de tiempo de las señales 
de salida y de entrada a un sistema o elemento equi vale también al cociente d

##Grafo

In [36]:
import re
with open('/content/Comunicacion.txt', 'r', encoding='utf-8') as archivo:
# Lee todo el contenido del archivo
  textg = archivo.read()

In [37]:
import spacy
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
from sklearn.metrics.pairwise import cosine_similarity
# Función para generar un resumen extractivo usando PageRank
def summarize(similarity_matrix, num_sentences=5):
# Crear un grafo a partir de la matriz de similitud
  nx_graph = nx.from_numpy_array(similarity_matrix)
# Aplicar PageRank al grafo
  scores = nx.pagerank(nx_graph)
# Ordenar las oraciones por su puntuación y seleccionar las mejores
  ranked_sentences = sorted(((scores[i], s) for i, s in enumerate(original_sentences)), reverse=True)
  return ' '.join([ranked_sentences[i][1] for i in range(num_sentences)])
# Cargar el modelo de spaCy
nlp = spacy.load('es_core_news_sm')

doc = nlp(textg)

# Lematizar y eliminar stopwords de cada oración
lemmatized_sentences = []
original_sentences = []
for sent in doc.sents:
  lemmatized_sentence = " ".join([token.lemma_ for token in sent if not token.is_stop and not token.is_punct])
  if lemmatized_sentence.strip() != '': # Asegurarse de que la oración no esté vacía
    lemmatized_sentences.append(lemmatized_sentence)
    original_sentences.append(str(sent).strip())
# Procesar las oraciones lematizadas con spaCy para obtener sus vectores
lemmatized_docs = [nlp(sent) for sent in lemmatized_sentences]
# Obtenemos una lista con los vectores de cada oración
sentence_vectors = [sent.vector for sent in lemmatized_docs]
# Crear una matriz de similitud entre las oraciones filtradas
similarity_matrix = cosine_similarity(sentence_vectors)
# Acortar las oraciones originales para usarlas como etiquetas en el gráfico
sentence_labels_short = []
for sent in original_sentences:
  if len(sent) > 50:
    sent = sent[:50] + '...'
  sentence_labels_short.append(sent)

# Generar resumen extractivo
res_comu = summarize(similarity_matrix, num_sentences=2)

In [38]:
def query_grafo(prompt, similarity_matrix=similarity_matrix, original_sentences=original_sentences):
    # Procesar la pregunta con spaCy
    question_doc = nlp(question)
    # Lematizar y eliminar stopwords de la pregunta
    lemmatized_question = " ".join([token.lemma_ for token in question_doc if not token.is_stop and not token.is_punct])
    # Calcular la similitud de la pregunta con cada oración
    question_vector = question_doc.vector
    similarities = cosine_similarity([question_vector], sentence_vectors)[0]
    # Ordenar las oraciones por su similitud con la pregunta
    ranked_sentences = sorted(((similarity, sent) for similarity, sent in zip(similarities, original_sentences)), reverse=True)
    # Devolver las oraciones más relevantes para la pregunta
    return [sent for similarity, sent in ranked_sentences[:3]]  # Devuelve las 3 oraciones más relevantes


In [39]:
# Hacer una pregunta
question = "¿cual es el voltaje de rs-232?"
relevant_sentences = query_grafo(question)
# Imprimir las oraciones relevantes
print("Oraciones relevantes para la pregunta:")
for sent in relevant_sentences:
    print("-", sent)

Oraciones relevantes para la pregunta:
- Ejemplo de voltajes: +12V para 0, -12V para 1.
- :

Direccion del Esclavo:

Cada dispositivo esclavo en un bus Modbus tiene una direccion única (usualmente de 1 a 247).
- Modbus es un protocolo de comunicacion utilizado en sistemas de automatizacion industrial para la transmision de datos entre dispositivos.


##Datos Tabulares

In [7]:
import pandas as pd

# Cargar el conjunto de datos
df = pd.read_csv('base_it.csv', delimiter=";")
df=df.dropna()


In [8]:
# Obtener los valores únicos de la columna 'Marca', 'tipo' y 'Zona'
marcas = df['Marca'].unique()
tipos = df['tipo'].unique()
zonas = df['Zona'].unique()

# Inicializar un diccionario vacío
dicc = {'marca': list(marcas), 'tipo': list(tipos), 'zona': list(zonas)}

# Imprimir el diccionario
print(dicc)

{'marca': ['cisco', 'rockwell', 'emerson', 'ruggedcom'], 'tipo': ['switch2', 'hub', 'router', 'switch3', 'repetidor'], 'zona': ['zonas_a', 'zonas_b', 'zonas_d', 'zonas_c']}


In [9]:
# Convertir el DataFrame en str
def dataframe_to_string(df):
    result = ""
    for index, row in df.iterrows():
        result += f"Marca: {row['Marca']} "
        result += f"Tipo: {row['tipo']} "
        result += f"Ventas: {row['ventas']} "
        result += f"Zona: {row['Zona']},"
        result += "\n"

    return result


In [27]:
def query_dataframe(query):
    # Inicializar la lista de claves encontradas como vacía
    claves_encontradas = []
    # Convertir a minúsculas
    query = query.lower()
    # Iterar sobre los elementos del diccionario
    for clave, valores in dicc.items():
        for palabra in query.split():  # Dividir la frase en palabras
            if palabra in valores:
                # Si se encuentra, almacenar la clave correspondiente
                claves_encontradas.append((clave, palabra))

    # Inicializar el contexto
    contexto = None

    # Realizar la consulta con las variables filtradas
    df_query = df.copy()

    if claves_encontradas:
        for clave, palabra in claves_encontradas:
          if clave == 'marca':
            df_query = df_query.query(f"Marca == '{palabra}'")
          elif clave == 'tipo':
            df_query = df_query.query(f"tipo == '{palabra}'")
          elif clave == 'zona':
            df_query = df_query.query(f"Zona == '{palabra}'")
        for clave in claves_encontradas:
          if clave == 'tipo':
            df_query = df_query.query(f"tipo == '{palabra}'")
          elif clave == 'zona':
            df_query = df_query.query(f"Zona == '{palabra}'")
        for clave in claves_encontradas:
          if clave == 'zona':
            df_query = df_query.query(f"Zona == '{palabra}'")

            # Convertir el DataFrame filtrado en contexto
        contexto = dataframe_to_string(df_query)
           # break  # Detener el bucle después de encontrar la primera coincidencia

    return contexto


In [31]:
contexto_data = query_dataframe('cuantos router cisco se vendieron?')
print(contexto_data)

Marca: cisco Tipo: router Ventas: 42.0 Zona: zonas_a,
Marca: cisco Tipo: router Ventas: 19.0 Zona: zonas_d,
Marca: cisco Tipo: router Ventas: 53.0 Zona: zonas_c,
Marca: cisco Tipo: router Ventas: 64.0 Zona: zonas_b,



##Clasificador

In [40]:
# !pip install transformers
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import nltk
from transformers import BertTokenizer, BertModel
import torch
import numpy as np


# Cargar el tokenizador y modelo preentrenado de BERT para español
model_name = 'dccuchile/bert-base-spanish-wwm-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

def get_bert_embeddings(texts):
    """Función para obtener los embeddings de BERT para una lista de textos."""
    embeddings = []
    for text in texts:
        inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
        # Usamos el embedding del token [CLS] como la representación del texto
        embeddings.append(outputs.last_hidden_state[0][0].numpy())
    return np.array(embeddings)

# Descargamos los stopwords que necesitaremos luego
nltk.download('stopwords')
from nltk.corpus import stopwords

# Obtenemos las stopwords para español
spanish_stop_words = stopwords.words('spanish')

labels = [(0, "instrumentacion"), (1, "comunicacion"), (2, "stock")]

dataset = []
# textos de "instrumentacion"
dataset.append((0, "Un sensor tipo transmisor."))
dataset.append((0, "Medicion de presion, temperatura."))
dataset.append((0, "el Spam es un rango."))
dataset.append((0, "rango de trabajo puede ser de 4ma - 20 ma."))

# textos de "comunicacion"
dataset.append((1, "Modbus es un protocolo de comunicacion y puede ser rtu o ascci."))
dataset.append((1, "Dos grandes protocolos son rs232 y rs485."))
dataset.append((1, "las redes industriales son fundamentales en el funcionamiento."))
dataset.append((1, "los datos pueden ser read analog imput."))


# textos de "stock"
dataset.append((2, "Las ventas de switchs cisco en zona a fueron de 46."))
dataset.append((2, "un hub tuvo mas venta que un router."))
dataset.append((2, "las zonas con mas venta fueron la zona a y la zona c."))
dataset.append((2, "los router en stock son 20."))
dataset.append((2, "zona a zona d"))
dataset.append((2, "ventas de switch2"))



# Preparar X e y
X = [text.lower() for label, text in dataset]
y = [label for label, text in dataset]

# División del dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Obtenemos los embeddings de BERT para los conjuntos de entrenamiento y prueba
X_train_vectorized = get_bert_embeddings(X_train)
X_test_vectorized = get_bert_embeddings(X_test)

# Creación y entrenamiento del modelo de Regresión Logística Multinomial
modelo_LR = LogisticRegression(max_iter=1000, multi_class='multinomial', solver='lbfgs')
modelo_LR.fit(X_train_vectorized, y_train)

# Evaluación del modelo de Regresión Logística
y_pred_LR = modelo_LR.predict(X_test_vectorized)
acc_LR = accuracy_score(y_test, y_pred_LR)
report_LR = classification_report(y_test, y_pred_LR, zero_division=1)

# Nuevas frases para clasificar
new_phrases = [
    "El rango del sensor varia.",
    "El modbus tiene listas.",
    "¿que ventas tuvo en la zona b?.",
]

# Preprocesamiento y vectorización de las nuevas frases
new_phrases_lower = [text.lower() for text in new_phrases]
new_phrases_vectorized = get_bert_embeddings(new_phrases_lower)

# Haciendo predicciones con el modelo entrenado
new_predictions = modelo_LR.predict(new_phrases_vectorized)

# Mostrando las predicciones junto con las frases
for text, label in zip(new_phrases, new_predictions):
    print(f"Texto: '{text}'")
    print(f"Clasificación predicha: {labels[label][1]}\n")


tokenizer_config.json:   0%|          | 0.00/364 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/480k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/648 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Texto: 'El rango del sensor varia.'
Clasificación predicha: instrumentacion

Texto: 'El modbus tiene listas.'
Clasificación predicha: comunicacion

Texto: '¿que ventas tuvo en la zona b?.'
Clasificación predicha: stock



In [63]:
#ejecutar el tag
def tag(prompt):
  new_phrases_lower = [text.lower() for text in prompt]
  new_phrases_vectorized = get_bert_embeddings(new_phrases_lower)
  # Predicciones con el modelo entrenado
  new_prediction = modelo_LR.predict(new_phrases_vectorized)
  tag= ({labels[new_prediction[0]][1]})
  etiquet = ', '.join(tag)
  return etiquet

##RAG

In [42]:
from llama_index import ServiceContext
from llama_index import VectorStoreIndex, SimpleDirectoryReader
from jinja2 import Template
import requests
from decouple import config
from llama_index.storage.storage_context import StorageContext
from llama_index.vector_stores.chroma import ChromaVectorStore

In [83]:
def contexto(etiqueta, prompt):
    if str(etiqueta) == "instrumentacion":
      chroma = query_chroma(prompt)
      return (chroma )#"Base de datos vectorial"
    elif str(etiqueta) == "comunicacion":
      grafo= query_grafo(prompt)
      return (grafo)#'Base de datos de grafos'
    elif str(etiqueta) == "stock":
      #dataframe= dataframe_to_string( prompt)
      dataframe = query_dataframe(prompt)
      return (dataframe)# 'Dataframe'


In [44]:
def zephyr_instruct_template(messages, add_generation_prompt=True):
    # Definir la plantilla Jinja
    template_str = "{% for message in messages %}"
    template_str += "{% if message['role'] == 'user' %}"
    template_str += "<|user|>{{ message['content'] }}</s>\n"
    template_str += "{% elif message['role'] == 'assistant' %}"
    template_str += "<|assistant|>{{ message['content'] }}</s>\n"
    template_str += "{% elif message['role'] == 'system' %}"
    template_str += "<|system|>{{ message['content'] }}</s>\n"
    template_str += "{% else %}"
    template_str += "<|unknown|>{{ message['content'] }}</s>\n"
    template_str += "{% endif %}"
    template_str += "{% endfor %}"
    template_str += "{% if add_generation_prompt %}"
    template_str += "<|assistant|>\n"
    template_str += "{% endif %}"

    # Crear un objeto de plantilla con la cadena de plantilla
    template = Template(template_str)

    # Renderizar la plantilla con los mensajes proporcionados
    return template.render(messages=messages, add_generation_prompt=add_generation_prompt)


# Aquí hacemos la llamada el modelo
def generate_answer(prompt: str, max_new_tokens: int = 768) -> None:
    try:
        # Tu clave API de Hugging Face
        api_key = config('HUGGINGFACE_TOKEN')

        # URL de la API de Hugging Face para la generación de texto
        api_url = "https://api-inference.huggingface.co/models/HuggingFaceH4/zephyr-7b-beta"

        # Cabeceras para la solicitud
        headers = {"Authorization": f"Bearer {api_key}"}

        # Datos para enviar en la solicitud POST
        # Sobre los parámetros: https://huggingface.co/docs/transformers/main_classes/text_generation
        data = {
            "inputs": prompt,
            "parameters": {
                "max_new_tokens": max_new_tokens,
                "temperature": 0.7,
                "top_k": 50,
                "top_p": 0.95
            }
        }

        # Realizamos la solicitud POST
        response = requests.post(api_url, headers=headers, json=data)

        # Extraer respuesta
        respuesta = response.json()[0]["generated_text"][len(prompt):]
        return respuesta

    except Exception as e:
        print(f"An error occurred: {e}")

# Esta función prepara el prompt en estilo QA
def prepare_prompt(query_str: str, context_str: list):
  TEXT_QA_PROMPT_TMPL = (
      "La información de contexto es la siguiente:\n"
      "---------------------\n"
      "{context_str}\n"
      "---------------------\n"
      "Dada la información de contexto anterior, y sin utilizar conocimiento previo, responde la siguiente pregunta.\n"
      "Pregunta: {query_str}\n"
      "Respuesta: "
  )

  messages = [
      {
          "role": "system",
          "content": "Eres un asistente útil que siempre responde con respuestas veraces, útiles y basadas en hechos.",
      },
      {"role": "user", "content": TEXT_QA_PROMPT_TMPL.format(context_str=context_str, query_str=query_str)},
  ]

  final_prompt = zephyr_instruct_template(messages)
  return final_prompt


In [85]:
print('Realizando llamada a HuggingFace para generar respuestas...\n')
queries = [
'¿Que rango de miliamperes trabaja un sensor?',
'¿Que es Modbus?',
'¿Cuantas ventas tuvo el router cisco?'
]
for query_str in queries:
# Traemos los documentos más relevantes para la consulta
  etiqueta = tag(queries)
  contexto_data = contexto(etiqueta, query_str)
  #contexto = retriever.retrieve(query_str)
  final_prompt = prepare_prompt(query_str, contexto_data)
  print('Pregunta:', query_str)
  print('Respuesta:')
  print(generate_answer(final_prompt))
  print('-------------------------------------------------------')


Realizando llamada a HuggingFace para generar respuestas...

Pregunta: ¿Que rango de miliamperes trabaja un sensor?
Respuesta:
El rango de miliamperes que trabaja un sensor varía ampliamente según el tipo de sensor. Algunos sensores de corriente eléctrica pueden medir miliamperes desde 0 hasta 20 mA, mientras que otros sensores de presión o temperatura no necesitan medir corrientes y por tanto, su rango de miliamperes es 0 mA. Es importante consultar el datosheet del sensor específico para conocer su rango de miliamperes de entrada y salida.
-------------------------------------------------------
Pregunta: ¿Que es Modbus?
Respuesta:
Modbus es un protocolo de comunicación abierto y de código abierto utilizado en la automatización industrial y el control de procesos. Permite la interconexión de dispositivos distintos y de diferentes fabricantes, ya que es independiente de la marca y del fabricante. El protocolo define la forma de transferencia de datos entre un controlador de sistema y d